        team1     team2  p_team1_wins  score1  score2  result  \
0      Chievo  Juventus           0.5       2       3       0   
1       Lazio    Napoli           0.5       1       2       0   
2      Torino      Roma           0.5       0       1       0   
3    Sassuolo     Inter           0.5       1       0       1   
4      Empoli  Cagliari           0.5       2       0       1   
..        ...       ...           ...     ...     ...     ...   
267      Spal     Milan           NaN       2       3       0   
268      Roma     Parma           NaN       2       1       1   
269     Inter    Empoli           NaN       2       1       1   
270  Cagliari   Udinese           NaN       1       2       0   
271  Atalanta  Sassuolo           NaN       3       1       1   

     team1_updated_mean  team2_updated_mean  
0             20.013221           29.986779  
1             20.013221           29.986779  
2             20.013221           29.986779  
3             29.986779           2

C:\Users\Berhane\AppData\Local\Temp\ipykernel_5816\3793477129.py:49: RuntimeWarning: divide by zero encountered in scalar divide
  outcome_mean = mean_diff + (norm.pdf(-mean_diff / std_diff) / (1 - norm.cdf(mean_diff / std_diff))) * std_diff
C:\Users\Berhane\AppData\Local\Temp\ipykernel_5816\3793477129.py:19: RuntimeWarning: invalid value encountered in scalar subtract
  mu_diff = mu1 - mu2
C:\Users\Berhane\AppData\Local\Temp\ipykernel_5816\3793477129.py:44: RuntimeWarning: invalid value encountered in scalar subtract
  mean_diff = skill1['mean'] - skill2['mean']
C:\Users\Berhane\AppData\Local\Temp\ipykernel_5816\3793477129.py:52: RuntimeWarning: invalid value encountered in scalar divide
  outcome_mean = mean_diff - (norm.pdf(-mean_diff / std_diff) / (norm.cdf(mean_diff / std_diff))) * std_diff


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm, norm

# Functions for conditional skills, outcomes, and marginal p(y = 1)

def conditional_skills(mu_s1, mu_s2, sigma_s1, sigma_s2, t, outcome_var=2):
    mean_s = np.array([mu_s1, mu_s2])
    cov_s = np.diag([sigma_s1**2, sigma_s2**2])
    outcome_mean = mu_s1 - mu_s2
    outcome_var = sigma_s1**2 + sigma_s2**2 + outcome_var

    mean_cond = mean_s + (t - outcome_mean) / outcome_var * np.array([sigma_s1**2, -sigma_s2**2])
    cov_cond = cov_s - np.outer([sigma_s1**2, -sigma_s2**2], [sigma_s1**2, -sigma_s2**2]) / outcome_var
    print(mean_cond)
    return mean_cond, cov_cond

def conditional_outcome(mu_s1, mu_s2, sigma_s1, sigma_s2, result, outcome_var=2):
    mean_diff = mu_s1 - mu_s2
    std_diff = np.sqrt(sigma_s1**2 + sigma_s2**2 + outcome_var)
    if result == 1:
        a, b = 0, np.inf
    else:
        a, b = -np.inf, 0

    trunc_gauss_dist = truncnorm(a=(a - mean_diff) / std_diff, b=(b - mean_diff) / std_diff, loc=mean_diff, scale=std_diff)
    
    return trunc_gauss_dist.mean(), trunc_gauss_dist.var()

def marginal_p_y1(mu_s1, mu_s2, sigma_s1, sigma_s2, sigma_epsilon):
    mu_diff = mu_s1 - mu_s2
    sigma_diff = np.sqrt(sigma_s1**2 + sigma_s2**2 + sigma_epsilon**2)
    p_y1 = 1 - norm.cdf(0, loc=mu_diff, scale=sigma_diff)
    return p_y1

# Load the dataset
file_path = 'SerieA.csv'
df = pd.read_csv(file_path)

# Initialize team skills
team_skills = {team: {'mean': 25, 'std': 8.33} for team in pd.concat([df['team1'], df['team2']]).unique()}

# Set noise variance
sigma_epsilon = 2  # Experiment with different values

# Learning rate for updating skills
alpha = 0.5

# List for results
results = []

# Loop through each match
for idx, row in df.iterrows():
    team1 = row['team1']
    team2 = row['team2']
    score1 = row['score1']
    score2 = row['score2']

    # Get team skill means and stddevs
    mu_s1 = team_skills[team1]['mean']
    mu_s2 = team_skills[team2]['mean']
    sigma_s1 = team_skills[team1]['std']
    sigma_s2 = team_skills[team2]['std']

    # Determine result
    result = 1 if score1 > score2 else 0
    
    # Compute conditional skills
    mean_cond, cov_cond = conditional_skills(mu_s1, mu_s2, sigma_s1, sigma_s2, result)

    # Compute conditional outcome distribution
    mean_trunc, var_trunc = conditional_outcome(mu_s1, mu_s2, sigma_s1, sigma_s2, result)

    # Compute marginal probability that team 1 wins
    p_team1_wins = marginal_p_y1(mu_s1, mu_s2, sigma_s1, sigma_s2, sigma_epsilon)

    # Update team skills
    team_skills[team1]['mean'] += alpha * (mean_cond[0] - mu_s1)
    team_skills[team2]['mean'] += alpha * (mean_cond[1] - mu_s2)

    # Append results
    results.append({
        'team1': team1,
        'team2': team2,
        'score1': score1,
        'score2': score2,
        'p_team1_wins': p_team1_wins
    })

# Convert to DataFrame and print results
results_df = pd.DataFrame(results)
print(results_df.head())
